In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import svm
from sklearn.model_selection import train_test_split

drive.mount('/content/gdrive')

Mounted at /content/gdrive


### Importando e explorando os dados

In [2]:
data = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_train.csv', sep=',')
col_names=[]
for i in range(len(data.columns)):
  col_names.append(str(i))
print(len(data.columns))
print(col_names)

data = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_train.csv', sep=',', names=col_names)


##Importando e explorando os dados

## TREINO
data_train = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_train.csv', sep=',')
col_names=[]
for i in range(len(data_train.columns)):
  col_names.append(str(i))
print(len(data_train.columns))
print(col_names)

data_train = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_train.csv', names=col_names)

## VALIDAÇÃO
data_val = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_val.csv', sep=',')
col_names=[]
for i in range(len(data_val.columns)):
  col_names.append(str(i))
print(len(data_val.columns))
print(col_names)

data_val = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_val.csv', names=col_names)

## TESTE
data_test = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_test.csv', sep=',')
col_names=[]
for i in range(len(data_test.columns)):
  col_names.append(str(i))
print(len(data_test.columns))
print(col_names)

data_test = pd.read_csv('/content/gdrive/MyDrive/Data/output_densenet_test.csv', names=col_names)


# Visualização dos dados 
print(data_train.shape)
data_train.head(10)

1025
['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '122', '123', '124', '125', '126', '127', '128', '129', '130', '131', '132', '133', '134', '135', '136', '137', '138', '139', '140', '141', '142', '143', '144', '145', '146', '147', '148', '149', '150', '151', '152', '153', '154', '155', '156', '157'

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024
0,0,0.000944,0.002661,0.012668,0.003094,0.038160,0.868616,0.000455,0.001779,0.285878,0.000020,0.003134,0.026718,0.0,0.026460,0.087229,0.0,0.000124,1.342344,0.003076,0.0,0.000368,0.001668,0.000525,0.000547,0.000788,0.003420,0.001687,0.040933,0.006250,0.003071,0.014719,0.001613,0.017109,0.000383,0.356637,0.001470,0.002454,0.081445,0.006500,...,4.327474,0.367923,1.954979,0.010795,0.406310,0.020173,0.079811,0.108834,1.016049,1.575079,4.842887,4.391789,4.852630,1.250430,1.619293,0.000000,0.145815,1.547524,0.191429,3.549697,0.163710,0.347341,0.080645,0.429869,1.151778,0.384554,3.742492,0.250907,0.158062,1.131968,0.285219,5.397420,0.145374,1.540241,1.922522,4.256811,0.017882,0.100336,1.367321,1.647147
1,0,0.000131,0.000930,0.006151,0.004051,0.080557,0.832400,0.000479,0.001071,0.118720,0.000101,0.005330,0.018502,0.0,0.028216,0.042158,0.0,0.000143,0.886432,0.001201,0.0,0.000351,0.001943,0.000250,0.000676,0.000286,0.002032,0.001585,0.031826,0.003988,0.004466,0.014574,0.001199,0.022001,0.000179,0.307661,0.002835,0.004195,0.083368,0.007170,...,2.130120,0.092624,2.506947,0.938912,0.038233,0.110226,0.000000,1.783702,1.357471,0.978679,4.052362,3.300156,5.589107,0.677885,1.525957,0.048105,0.370356,1.464557,0.161798,5.449239,0.137261,0.084729,0.097049,0.022151,0.322784,0.551517,2.676725,0.344216,0.131846,1.021914,0.067933,1.863616,0.000000,0.681680,2.798108,4.224206,0.005430,2.330088,0.175864,0.714172
2,1,0.001063,0.002809,0.005221,0.005379,0.036074,1.305328,0.000706,0.002928,0.286010,0.000009,0.004626,0.012053,0.0,0.045496,0.071221,0.0,0.000178,1.734423,0.003203,0.0,0.000320,0.002209,0.000300,0.001266,0.000453,0.003918,0.001467,0.051461,0.013850,0.000609,0.012049,0.001437,0.012310,0.000232,0.423529,0.003007,0.005714,0.073672,0.007671,...,1.455053,0.791326,3.538281,1.352926,0.562719,0.441343,3.215886,1.315869,8.404341,1.996801,1.043715,0.352527,1.419540,3.219539,0.702554,3.237000,0.850772,0.197373,4.618689,2.221408,0.088727,1.677526,8.516278,0.083652,3.081773,13.785527,1.174299,3.315265,1.469981,5.605446,0.000000,1.163371,1.494781,1.508267,1.488753,0.070286,2.081975,2.731160,0.000000,0.112964
3,1,0.001046,0.001331,0.002944,0.003391,0.026359,0.944276,0.000494,0.001258,0.116351,0.000020,0.005245,0.025591,0.0,0.037319,0.111816,0.0,0.000197,2.050375,0.004945,0.0,0.000994,0.001592,0.000158,0.001269,0.001354,0.003266,0.000668,0.028938,0.013980,0.000722,0.016431,0.001530,0.016292,0.000084,0.544158,0.002674,0.007691,0.085426,0.009555,...,1.603098,1.230846,3.044707,0.790974,0.745773,1.260065,2.231487,2.299075,8.264530,0.044334,0.492695,0.943517,1.239837,5.823122,0.165037,2.727797,3.014661,0.247995,5.276898,0.161920,0.122879,0.705785,6.947849,0.012496,3.499351,7.579140,0.766171,3.341328,0.575052,11.992809,0.017204,0.034326,2.761541,2.596450,0.585777,0.207722,0.796570,4.714148,0.000000,0.895667
4,1,0.000906,0.001818,0.002844,0.001040,0.045158,0.312737,0.000319,0.000702,0.049549,0.000000,0.006377,0.021475,0.0,0.010560,0.114066,0.0,0.000073,1.539964,0.004262,0.0,0.000119,0.002659,0.000586,0.001279,0.002384,0.001284,0.001774,0.026849,0.006699,0.002508,0.016806,0.001484,0.031585,0.000115,0.779026,0.004616,0.004335,0.034269,0.003487,...,0.692682,0.365230,2.936770,1.121329,2.291025,0.424125,2.728684,0.309267,5.641420,0.404378,1.429571,0.144882,0.174041,2.552285,0.151432,2.035253,1.392122,0.124659,2.733172,0.787355,1.358319,0.583778,0.306336,0.088490,1.427957,4.401023,0.235291,0.364074,2.309863,3.994056,0.003943,0.148177,1.537525,0.537371,0.091987,0.209803,0.577082,2.098766,0.000262,0.153663
5,1,0.000534,0.002330,0.001346,0.003003,0.099872,0.826041,0.000188,0.001511,0.264414,0.000008,0.004844,0.015948,0.0,0.023319,0.118277,0.0,0.000005,1.627564,0.002998,0.0,0.000839,

Iremos plotar abaixo a distribuição das classes em histograma

In [ ]:
plt.style.use('ggplot')
plt.hist(data[1], bins=7)

Pela descrição do dataset, o tipo de vidro é de 1 a 7, porém não existem dados para o tipo 4


In [ ]:
data.describe() 

In [ ]:
data.info()

Pelas informações do dataset vemos que não existe nenhum dado nulo na base, portanto não precisamos realizar tratamento de nulos

Iremos plotar os gráficos dos pares de variáveis e da sua distribuição individual

In [ ]:
variaveis_pertinentes = data
sns.set(style="ticks", color_codes=True)
plt.style.use('ggplot')
# Plotando a distribuição em pares das variáveis pertinentes e seus histogramas
plt.figure(figsize=(8,8))
sns.pairplot(variaveis_pertinentes,palette='coolwarm', corner=True, kind="reg", diag_kind="kde")
plt.show()


Analisando os gráficos as únicas variáveis que parecem ter correlação visivel uma com a outra são a RI e a Ca. Também é possível concluir que a distribuição de todas as variáveis segue um padrão de ter picos em poucos valores e ocorrência baixa em todos os outros, menos a variável Ba, que possui vários picos e vales.

### Matriz de correlação

In [ ]:
# Visualizando a matriz de correlação
corr = variaveis_pertinentes.corr()
plt.figure(figsize=(10,10))
sns.heatmap(corr, cbar = True,  square = True, annot=True, xticklabels= corr.columns.values, yticklabels= corr.columns.values, cmap= 'coolwarm')
plt.show()

print(corr) # Verificando a correlão entre as variáveis

Plotando a matriz de correlação confirmamos que a única correlação forte é a de Ca com RI, porém também visualizamos uma correlação média-forte entre Mg e Type.

In [ ]:
corr_min = abs(corr).values.min()
np.where(abs(corr)==corr_min)

In [ ]:
variaveis_pertinentes.columns[[0,7]]

As duas variáveis com menor correlação são RI e Ba, seguidos de Ca e Type, elas são as que mais se aproximam de zero.

### Dividindo o dataset em X e y (data e target)

In [3]:
## Dividindo o dataset em X e y (data e target)

#y = data[['0']]
#X = data.drop(['0',],1)
#print(data.shape, X.shape, y.shape)

y_train = data_train[['0']]
X_train = data_train.drop(['0',],1)
print(data_train.shape, X_train.shape, y_train.shape)

y_val = data_val[['0']]
X_val = data_val.drop(['0',],1)
print(data_val.shape, X_val.shape, y_val.shape)

y_test = data_test[['0']]
X_test = data_test.drop(['0',],1)
print(data_test.shape, X_test.shape, y_test.shape)

(4208, 1025) (4208, 1024) (4208, 1)
(1100, 1025) (1100, 1024) (1100, 1)
(391, 1025) (391, 1024) (391, 1)


### Modelo SVM


Definir uma função que avaliar o modelo SVM com kernel variando como candidato, usando validação cruzada estratificada repetida de 5 vezes e, em seguida, retornar uma lista de pontuações calculadas no modelo para cada dobra e repetição.

In [4]:
# baseline model and test harness for the glass identification dataset

# Avaliar o modelo
def evaluate_model(X_train, y_train, model):
	# Utilizar o RepeatedStratified k-fold 5x10
	cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=1)
	# Avaliando o modelo
	scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1)
	return scores

Definimos uma função get_models() que recebe 4 tipos de kernels diferentes para SVM e os retorna com alguns parâmetros pré-definidos.

In [5]:
# Definindo a priori, qual o modelo de referencia
# define models to test
def get_models():
  C = 1.0  # SVM regularization parameter
  models = (svm.SVC(kernel='linear', C=C),
          svm.LinearSVC(C=C, max_iter=10000),
          svm.SVC(kernel='rbf', gamma=0.7, C=C),
          svm.SVC(kernel='poly', degree=5, gamma='auto', C=C))
  names = ('SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 5) kernel')
  return models, names


Podemos então avaliar o modelo chamando a função evaluate_model() em seguida salvar os scores na lista results que relata a média e o desvio padrão dos resultados.

In [6]:
# Definindo modelos
models, names = get_models()
results = list()
# Avaliando cada modelo
for i in range(len(models)):
	# Avaliando o modelo e armazenando os resultados
	scores = evaluate_model(X_train, y_train, models[i])
	results.append(scores)
	# Resumo das performances
	print('>%s média: %.3f ( desvio padrão: %.3f)' %(names[i], np.mean(scores), np.std(scores)))

>SVC with linear kernel média: 0.984 ( desvio padrão: 0.004)
>LinearSVC (linear kernel) média: 0.984 ( desvio padrão: 0.004)
>SVC with RBF kernel média: 0.595 ( desvio padrão: 0.001)
>SVC with polynomial (degree 5) kernel média: 0.984 ( desvio padrão: 0.004)


### Dividindo o conjunto de dados em treinamento e teste e realizando o Repeated K-Fold com 5x10

In [11]:
## Dividindo o conjunto de dados em treinamento e teste e realizando o Repeated K-Fold com 5x10

from sklearn.metrics import accuracy_score

#y = data[["0"]].values
#X = data.drop(['0',],1).values

svckernellinear = []
linearsvc = []
svcrbf = []
poly = []

svckernellinear_train = []
linearsvc_train = []
svcrbf_train = []
poly_train = []

# Usando os mesmos modelos
models, names = get_models()

skf = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=2673512)
#for train_index, test_index in skf.split(X_train, y_train):
  #print("TRAIN:", train_index, "TEST:", test_index)
  #X_train, X_test = X[train_index], X[test_index]
  #y_train, y_test = y[train_index], y[test_index]

index = 0
names = ['SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 5) kernel']
for mod in models:
  nm = names[index]
  mod.fit(X_train, y_train)
  y_pred = mod.predict(X_test)
  class_report = accuracy_score(y_test, y_pred) 

  y_pred1 = mod.predict(X_train)
  class_report1 = accuracy_score(y_train, y_pred1) 

  if nm == 'SVC with linear kernel':
    svckernellinear.append(class_report)
    svckernellinear_train.append(class_report1)
  elif nm == 'LinearSVC (linear kernel)':
    linearsvc.append(class_report)
    linearsvc_train.append(class_report1)
  elif nm == 'SVC with RBF kernel':
    svcrbf.append(class_report) 
    svcrbf_train.append(class_report1) 
  else:
    poly.append(class_report)
    poly_train.append(class_report1)

  index = index +1


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

## Avaliando a acurácia dos modelos

In [12]:
# Modelo com melhor acurácia do SVC with linear kernel
print("SVC with linear kernel")
print("---- Conjunto teste ----")
print("Melhor acurácia: {}".format(max(svckernellinear)))
print("Média da acurácia: {}".format(np.mean(svckernellinear)))

print("---- Conjunto treino ----")
print("Melhor acurácia: {}".format(max(svckernellinear_train)))
print("Média da acurácia: {}".format(np.mean(svckernellinear_train)))


SVC with linear kernel
---- Conjunto teste ----
Melhor acurácia: 0.8951406649616368
Média da acurácia: 0.8951406649616368
---- Conjunto treino ----
Melhor acurácia: 1.0
Média da acurácia: 1.0


In [13]:
# Modelo com melhor acurácia do LinearSVC (linear kernel)
print("LinearSVC (linear kernel)")
print("---- Conjunto teste ----")
print("Melhor acurácia: {}".format(max(linearsvc)))
print("Média da acurácia: {}".format(np.mean(linearsvc)))

print("---- Conjunto treino ----")
print("Melhor acurácia: {}".format(max(linearsvc_train)))
print("Média da acurácia: {}".format(np.mean(linearsvc_train)))

LinearSVC (linear kernel)
---- Conjunto teste ----
Melhor acurácia: 0.8900255754475703
Média da acurácia: 0.8900255754475703
---- Conjunto treino ----
Melhor acurácia: 1.0
Média da acurácia: 1.0


In [14]:
# Modelo com melhor acurácia do SVC with RBF kernel

print("SVC with RBF kernel")
print("---- Conjunto teste ----")
print("Melhor acurácia: {}".format(max(svcrbf)))
print("Média da acurácia: {}".format(np.mean(svcrbf)))

print("---- Conjunto treino ----")
print("Melhor acurácia: {}".format(max(svcrbf_train)))
print("Média da acurácia: {}".format(np.mean(svcrbf_train)))

SVC with RBF kernel
---- Conjunto teste ----
Melhor acurácia: 0.2557544757033248
Média da acurácia: 0.2557544757033248
---- Conjunto treino ----
Melhor acurácia: 1.0
Média da acurácia: 1.0


In [15]:
# Modelo com melhor acurácia do SVC with polynomial (degree 5) kernel

print("SVC with polynomial (degree 5) kernel")
print("---- Conjunto teste ----")
print("Melhor acurácia: {}".format(max(poly)))
print("Média da acurácia: {}".format(np.mean(poly)))

print("---- Conjunto treino ----")
print("Melhor acurácia: {}".format(max(poly_train)))
print("Média da acurácia: {}".format(np.mean(poly_train)))

SVC with polynomial (degree 5) kernel
---- Conjunto teste ----
Melhor acurácia: 0.9002557544757033
Média da acurácia: 0.9002557544757033
---- Conjunto treino ----
Melhor acurácia: 0.9995247148288974
Média da acurácia: 0.9995247148288974
